In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pickle
import json

In [17]:
dtypes = {'cfips': str}
df = pd.read_csv('train.csv', dtype=dtypes)
df['cfips'] = df['cfips'].apply(lambda x: x.zfill(5))
df.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,01001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,01001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,01001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,01001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,01001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [18]:
# het rid of the date's day part
df['first_day_of_month'] = df['first_day_of_month'].apply(lambda x: x[:7])
df.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,01001,Autauga County,Alabama,2019-08,3.007682,1249
1,1001_2019-09-01,01001,Autauga County,Alabama,2019-09,2.884870,1198
2,1001_2019-10-01,01001,Autauga County,Alabama,2019-10,3.055843,1269
3,1001_2019-11-01,01001,Autauga County,Alabama,2019-11,2.993233,1243
4,1001_2019-12-01,01001,Autauga County,Alabama,2019-12,2.993233,1243


In [19]:
density_dates = df['first_day_of_month'].unique()
density_dates

array(['2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01',
       '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07',
       '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01',
       '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07',
       '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01',
       '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07',
       '2022-08', '2022-09', '2022-10'], dtype=object)

In [20]:
dates = np.array(['2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
       '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
       '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
       '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
       '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
       '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03', '2023-04', '2023-05'],
      dtype=object)


In [21]:
cfips_df = pd.read_csv('cfips.csv', dtype=dtypes)
cfips_df.head()

,cfips
0,01001
1,01003
2,01005
3,01007
4,01009


In [22]:
data = np.ones((len(dates), 1, len(cfips_df)))
data.shape      

(47, 1, 3142)

In [35]:
data = np.ones((len(dates), 1, len(cfips_df))) * np.nan

num_null = 0
for date_idx, date in enumerate(dates):
    if date not in density_dates:
        data[date_idx,:,:] = np.nan
        num_null += 1
    else:
        density_dated_df = df[df['first_day_of_month'] == date]
        cfips_merged_df = pd.merge(cfips_df, density_dated_df, on='cfips', how='left')
        panel_density_df = cfips_merged_df[['cfips', 'microbusiness_density']]
        panel_data = panel_density_df['microbusiness_density'].to_numpy()
        data[date_idx,:,:] = panel_data

print(f'Number of null dates: {num_null}')

Number of null dates: 8


In [36]:
data.shape

(47, 1, 3142)

In [37]:
data[:,:,0]

array([[      nan],
       [3.0076818],
       [2.8848701],
       [3.0558431],
       [2.9932332],
       [2.9932332],
       [2.96909  ],
       [2.9093256],
       [2.9332314],
       [3.0001674],
       [3.0049484],
       [3.0192919],
       [3.0838373],
       [3.174679 ],
       [3.2057564],
       [3.1938035],
       [3.0384164],
       [3.002558 ],
       [2.9472437],
       [3.1061056],
       [3.1440427],
       [3.2246592],
       [3.2270303],
       [3.2222881],
       [3.2104328],
       [3.2199171],
       [3.186722 ],
       [3.2033195],
       [3.2009485],
       [3.2863071],
       [3.2967808],
       [3.3344314],
       [3.3367846],
       [3.372082 ],
       [3.3132529],
       [3.3461974],
       [3.4379706],
       [3.4238517],
       [3.442677 ],
       [3.4638555],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan]])

## Looks right for Autauga

In [38]:
data_name = 'Given_Density'
data_description = {'feature_0' : 'microbusiness_density'}

# save data in pickle format
with open(f'BLOCKED_{data_name}_data.pkl', 'wb') as f:
    pickle.dump(data, f)

# save data in json format
with open(f'BLOCKED_{data_name}_data_description.json', 'w') as f:
    json.dump(data_description, f)